In [1]:
import pandas as pd
from database.market import Market
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
market = Market()
spec = ADatabase("spec")
umod = UniversalModeler()

In [3]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [4]:
factors = [str(x) for x in range(14)]

In [5]:
included_columns=[
    "year",
    "week",
    "ticker",
    "adjclose",
    "y"
]

In [6]:
included_columns.extend(factors)

In [7]:
training_sets = []

In [8]:
for ticker in sp500["ticker"].unique():
    try:
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        prices["year"] = [x.year for x in prices["date"]]
        prices["quarter"] = [x.quarter for x in prices["date"]]
        ticker_data = prices[prices["ticker"]==ticker]
        ticker_data.sort_values("date",ascending=True,inplace=True)
        ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
        ticker_data = ticker_data.groupby(["year","week"]).mean().reset_index()
        for i in range(14):
            ticker_data[str(i)] = ticker_data["adjclose"].shift(i)
        ticker_data["d1"] = ticker_data["adjclose"].pct_change(periods=1)
        ticker_data["d2"] = ticker_data["d1"].pct_change(periods=1)
        ticker_data["d3"] = ticker_data["d2"].pct_change(periods=1)
        ticker_data["rolling14"] = ticker_data["adjclose"].rolling(window=14).mean()
        ticker_data.dropna(inplace=True)
        ticker_data["ticker"] = ticker
        ticker_data["y"] = ticker_data["adjclose"].shift(-1)
        ticker_data = ticker_data.replace([np.inf, -np.inf], np.nan).dropna()
        ticker_data.dropna(inplace=True)
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except:
        continue

In [9]:
data = pd.concat(training_sets)

In [10]:
data = data.dropna()

In [11]:
spec.connect()
years = 4
for modeler in [umod]:
    for year in tqdm(range(2009,2023)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - years)].reset_index(drop=True)
        prediction_set = data[data["year"]==year].reset_index(drop=True)
        set_name = "sim"
        stuff = modeler.model(training_slice,prediction_set,factors)
        stuff["training_years"] = years
        spec.store(set_name,stuff)
spec.disconnect()


  0%|                                                                                                                                                             | 0/14 [00:00<?, ?it/s]

Epoch 1/10
2012/2012 [==============================] - 2s 752us/step - loss: 77.1475 - mean_absolute_percentage_error: 9.0433
Epoch 2/10
2012/2012 [==============================] - 1s 736us/step - loss: 28.4474 - mean_absolute_percentage_error: 6.9298
Epoch 3/10
2012/2012 [==============================] - 1s 735us/step - loss: 24.2801 - mean_absolute_percentage_error: 6.3560
Epoch 4/10
2012/2012 [==============================] - 2s 761us/step - loss: 24.2860 - mean_absolute_percentage_error: 6.2407
Epoch 5/10
2012/2012 [==============================] - 2s 762us/step - loss: 21.0680 - mean_absolute_percentage_error: 5.4089
Epoch 6/10
2012/2012 [==============================] - 2s 758us/step - loss: 23.3633 - mean_absolute_percentage_error: 5.6942
Epoch 7/10
2012/2012 [==============================] - 2s 759us/step - loss: 22.2354 - mean_absolute_percentage_error: 5.7466
Epoch 8/10
2012/2012 [==============================] - 2s 758us/step - loss: 20.6229 - mean_absolute_percentag

  7%|██████████▋                                                                                                                                          | 1/14 [01:01<13:18, 61.39s/it]

Epoch 1/10
2053/2053 [==============================] - 2s 746us/step - loss: 38.9610 - mean_absolute_percentage_error: 8.7095
Epoch 2/10
2053/2053 [==============================] - 2s 742us/step - loss: 28.8343 - mean_absolute_percentage_error: 7.5592
Epoch 3/10
2053/2053 [==============================] - 2s 742us/step - loss: 23.5254 - mean_absolute_percentage_error: 6.6559
Epoch 4/10
2053/2053 [==============================] - 2s 743us/step - loss: 19.8118 - mean_absolute_percentage_error: 6.2656
Epoch 5/10
2053/2053 [==============================] - 2s 741us/step - loss: 18.9273 - mean_absolute_percentage_error: 5.9387
Epoch 6/10
2053/2053 [==============================] - 2s 750us/step - loss: 16.8252 - mean_absolute_percentage_error: 5.8703
Epoch 7/10
2053/2053 [==============================] - 2s 746us/step - loss: 16.7386 - mean_absolute_percentage_error: 5.6743
Epoch 8/10
2053/2053 [==============================] - 2s 745us/step - loss: 16.5656 - mean_absolute_percentag

 14%|█████████████████████▎                                                                                                                               | 2/14 [02:02<12:14, 61.25s/it]

...

 86%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                     | 12/14 [13:38<02:23, 71.97s/it]

Epoch 1/10
2377/2377 [==============================] - 2s 756us/step - loss: 1030.3181 - mean_absolute_percentage_error: 8.6599
Epoch 2/10
2377/2377 [==============================] - 2s 796us/step - loss: 290.5408 - mean_absolute_percentage_error: 6.3674
Epoch 3/10
2377/2377 [==============================] - 2s 761us/step - loss: 292.7289 - mean_absolute_percentage_error: 5.9428
Epoch 4/10
2377/2377 [==============================] - 2s 759us/step - loss: 201.9640 - mean_absolute_percentage_error: 5.1564
Epoch 5/10
2377/2377 [==============================] - 2s 745us/step - loss: 214.1402 - mean_absolute_percentage_error: 5.1968
Epoch 6/10
2377/2377 [==============================] - 2s 797us/step - loss: 202.4704 - mean_absolute_percentage_error: 5.0852
Epoch 7/10
2377/2377 [==============================] - 2s 723us/step - loss: 241.5986 - mean_absolute_percentage_error: 5.1943
Epoch 8/10
2377/2377 [==============================] - 2s 717us/step - loss: 205.1389 - mean_absolute_

 93%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍          | 13/14 [14:54<01:13, 73.29s/it]

Epoch 1/10
2394/2394 [==============================] - 2s 719us/step - loss: 886.6486 - mean_absolute_percentage_error: 8.1182
Epoch 2/10
2394/2394 [==============================] - 2s 776us/step - loss: 426.2489 - mean_absolute_percentage_error: 6.3671
Epoch 3/10
2394/2394 [==============================] - 2s 713us/step - loss: 386.0768 - mean_absolute_percentage_error: 5.7646
Epoch 4/10
2394/2394 [==============================] - 2s 718us/step - loss: 366.5302 - mean_absolute_percentage_error: 5.5408
Epoch 5/10
2394/2394 [==============================] - 2s 721us/step - loss: 245.0331 - mean_absolute_percentage_error: 5.0596
Epoch 6/10
2394/2394 [==============================] - 2s 764us/step - loss: 273.3865 - mean_absolute_percentage_error: 5.0238
Epoch 7/10
2394/2394 [==============================] - 2s 733us/step - loss: 248.7941 - mean_absolute_percentage_error: 4.7673
Epoch 8/10
2394/2394 [==============================] - 2s 716us/step - loss: 218.9812 - mean_absolute_p

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [16:08<00:00, 69.20s/it]
